In [ ]:
import os
import pandas as pd
import json
from IPython.display import display
from urllib.request import urlopen
import urllib.request
from bs4 import BeautifulSoup as soup
import jieba.analyse as jaa

dataFolder = './medicalBeautySearchResult'
files = os.listdir(dataFolder)
master = pd.DataFrame()

for f in files:
  if f.endswith('.json'):
    # print(f)
    with open(os.path.join(dataFolder, f)) as ff:

      # read the text content of the .json file
      data = ff.read()

      # turn json data from str status to dict, so that we can use them as dict
      result = json.loads(data)

      # we only want the organic_results section

      # turn every list of dict into a dataframe
      df = pd.DataFrame(result['organic_results'])
      master = pd.concat([master, df])

In [ ]:
masterNoDup = master.drop_duplicates(subset=['title'], ignore_index=True)
len(masterNoDup)

794

In [ ]:
# HK
inHK = []
for i, link in enumerate(masterNoDup['displayed_link']):
  link_domain = link.split('›')[0]
  inHK.append('.hk' in link_domain)
  masterNoDup.loc[i,'inHK'] = ('hk' in link_domain)

hklinks = masterNoDup[masterNoDup['inHK']==True].reset_index(drop=True)
print(masterNoDup['inHK'].value_counts())

# TW
inTW = []
for i, link in enumerate(masterNoDup['displayed_link']):
  link_domain = link.split('›')[0]
  inTW.append('.tw' in link_domain)
  masterNoDup.loc[i,'inTW'] = ('tw' in link_domain)

twlinks = masterNoDup[masterNoDup['inTW']==True].reset_index(drop=True)
print(masterNoDup['inTW'].value_counts())

/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1684: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


False    598
True     196
Name: inHK, dtype: int64
False    657
True     137
Name: inTW, dtype: int64


HK:

In [ ]:
hk_keywords = []

for i, url in enumerate(hklinks['link']):

  ###
  # if i < 50:
  hdr = {'User-Agent':'Mozilla/5.0', 'Accept': 'text/html, application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8'}
  try:
    req = urllib.request.Request(url, headers=hdr)
    html_page = urllib.request.urlopen(req, timeout=2)
    html_soup = soup(html_page, "html.parser")
    html_soup

    htmlTagToKeep = ['title', 'p', 'h1', 'h2', 'h3', 'h4']
    allTags = html_soup.findAll(htmlTagToKeep)

    for tag in allTags:
      hk_keywords.append(tag.text)
      # print(tag.text)
    print(url)
  except:
    print('Failed: '+url)
  ###

len(hk_keywords)

https://www.hk01.com/%E5%81%A5%E5%BA%B7Easy/709985/%E6%89%93%E6%96%91%E5%BE%8C%E8%AD%B7%E7%90%86-%E6%BF%80%E5%85%89%E5%8E%BB%E6%96%91%E5%BE%8C%E7%9A%AE%E8%86%9A%E6%B3%9B%E7%B4%85%E4%B9%BE%E7%87%A5%E7%B5%90%E7%84%A6-4%E6%8B%9B%E6%94%B9%E5%96%84%E5%95%8F%E9%A1%8C%E5%8A%A0%E5%BC%B7%E6%95%88%E6%9E%9C
https://www.cosmopolitan.com.hk/beauty/post-laser-treatment-skincare-tips
https://bora.hk/%E7%9A%AE%E7%A7%92%E5%8E%BB%E6%96%91%E5%BE%8C%E8%AD%B7%E7%90%86
https://www.cleanskincare.com.hk/blog/2016/02/blog-post_18-8.html
https://20beauty.com.hk/%E8%84%AB%E6%AF%9B%E5%BE%8C%E8%AD%B7%E7%90%86
https://beauty.ulifestyle.com.hk/beauty/skincare/22845/%E9%86%AB%E7%BE%8E%E5%BF%83%E5%BE%97-%E8%90%AC%E5%85%83%E7%99%82%E7%A8%8B%E5%80%BC%E5%BE%97%E5%97%8E-%E6%BF%80%E5%85%89%E5%8E%BB%E6%96%91%E7%9A%84%E7%9C%9F%E5%AF%A6%E7%94%A8%E5%AE%B6-%E8%A6%AA%E8%BA%AB%E5%88%86%E4%BA%AB-%E8%A1%93%E5%BE%8C%E8%AD%B7%E7%90%86%E5%BF%83%E5%BE%97-%E8%A9%B3%E7%B4%B0%E8%A7%A3%E8%AA%AA%E6%89%93%E6%96%91%E7%9A%84%E4%B8%80%E4%BA%8C%

https://zh.zalora.com.hk/akarana-baby-laser-cut-basic-nursing-sleep-yoga-bra-for-breastfeeding-women-button-front-maternity-comfy-bralette-nude-6155482.html
https://www.carousell.com.hk/p/%E2%9C%A8%E7%A2%B3%E7%B2%89%E6%BF%80%E5%85%89%E8%AD%B7%E7%90%86-298-%E5%AD%B8%E7%94%9F-%E9%86%AB%E8%AD%B7%E7%85%A7%E6%B8%9B-100-1179957004/


https://gleneagles.hk/user_uploads/Information-Sheet-on-Post-Carbon-Dioxide-CO2-Laser-Chi_20171206.pdf
https://beauskinmedical.hk/medical-tips
https://perfectmen.hk/blog/hair-removal-tips-for-men/beard-removal/
Failed: https://www.healthlase.com.hk/
https://ibeauty.com.hk/mobile/article-inner/?id=4094&noad=
Failed: https://ladieschoice.com.hk/product/%E6%BF%80%E5%85%89%E7%A2%B3%E7%B2%89%E8%83%8C%E9%83%A8%E5%8E%BB%E5%8D%B0%E8%AD%B7%E7%90%86/
https://www.one-g.com.hk/helios-iii-laser-skin-care-treatment/
https://www.hklasereye.com/lasik
https://apmedical.hk/promotionlist/
https://www.mamame.com.hk/spectra
https://www.microsunhk.com/280844208858/micro-sun-
https://www.beautilike.hk/tc/product/c8-%E7%A2%B3%E7%B2%89%E6%BF%80%E5%85%89%E9%9D%A2%E9%83%A8%E8%AD%B7%E7%90%86/
https://www.elle.com.hk/beauty_and_health/all-you-need-to-know-about-laser-mole-removal
https://mobile.hksh.com/zh-hk/Ask%20Dr.HKSH/6
https://www.honeypot.com.hk/zh-Hant/treatments
https://joinbeauty.com.hk/after-care/
https

https://www.hkqf.gov.hk/filemanager/common/SCS_TP/be2.pdf
https://www.iskin.com.hk/zh-hant/%E7%9A%AE%E8%86%9A%E5%95%8F%E9%A1%8C/%E6%89%8B%E9%83%A8%E8%AD%B7%E7%90%86/
https://www.hk01.com/%E7%BE%8E%E5%AE%B9%E6%89%8B%E5%B8%B3/601451/%E7%BE%8E%E7%99%BD%E8%AD%B7%E8%86%9A-%E9%9B%B7%E5%B0%84%E6%BF%80%E5%85%89%E5%8E%BB%E6%96%91%E6%8F%9B%E8%86%9A%E6%87%B6%E4%BA%BA%E5%8C%85-%E6%95%88%E7%94%A8%E9%A2%A8%E9%9A%AA%E5%8F%8A%E8%A1%93%E5%BE%8C%E8%AD%B7%E7%90%86%E4%B8%80%E6%96%87%E7%9C%8B%E6%B8%85
https://www.epureskinhk.com/blogs/epure/jelly-masque-for-laser-treatment
https://oasis-group.com.hk/blog/%E4%BB%A5%E4%B8%8B5%E9%BB%9E%E9%81%BF%E5%85%8D%E7%9A%AE%E7%A7%92%E6%BF%80%E5%85%89%E5%8E%BB%E6%96%91%E5%BE%8C%E5%8F%8D%E9%BB%91/
https://perfectmen.hk/p9/
https://new-beauty.hk/blog/spot-removal-treatment/
https://hk-medicare.com/fractional-co2-laser/
https://www.retens.hk/medical-beauty/picosure/
https://hkbeautygirls.com/%E6%BF%80%E5%85%89%E5%8E%BB%E6%96%91/
https://blog.ulifestyle.com.hk/article/pingpin

https://www.med.hku.hk/healthedu/issue289/issue289.pdf
https://www.vigour.com.hk/post/_%E7%9A%AE%E7%A7%92%E5%8E%BB%E6%96%91
https://blog.beauty-place.com.hk/%E6%BF%80%E5%85%89%E5%8E%BB%E6%96%91/
https://onlux.com.hk/knowledge/%E6%BF%80%E5%85%89%E5%8E%BB%E6%96%91%E5%89%AF%E4%BD%9C%E7%94%A8
https://www.youbeauty.com.hk/tc/service_detail.php?cateid=13&id=22
https://medicalbeauty.com.hk/%E7%94%9A%E9%BA%BD%E6%98%AF%E7%9A%AE%E7%A7%92%E5%8E%BB%E6%96%91/
http://www.alumni.cuhk.edu.hk/magazine/sep98/html/p08-09.htm
https://www.healingbeam.com.hk/excel-v
Failed: https://www.metroradio.com.hk/Campaign/104/Skin_Lasercare/120929.html
https://www.3phk.com/article.asp?id=337
https://www.psmedical.com.hk/blog/pigmentation-removal-tips/difference-between-sun-sports-and-other-types-of-pigmentation/
https://www.miniso.hk/%E6%BF%80%E5%85%89%E5%8E%BB%E6%96%91%E9%86%AB%E7%94%9F%E6%94%B6%E8%B2%BB%E8%A9%B3%E7%B4%B0%E6%94%BB%E7%95%A5/
http://www.etnet.com.hk/www/tc/lifestyle/family/healthyd/48353
https://urban

https://www.sth.org.hk/upload/services/original/705520180412094749.pdf
https://hk.news.yahoo.com/%E7%9A%AE%E7%A7%92%E6%BF%80%E5%85%89%E5%8E%BB%E6%96%91%E6%AF%94%E5%82%B3%E7%B5%B1%E6%BF%80%E5%85%89%E5%8E%BB%E6%96%91%E6%9B%B4%E6%9C%89%E6%95%88-%E6%8A%8A%E6%8F%A1%E6%BF%80%E5%85%89%E5%8E%BB%E6%96%91%E5%BE%8C%E8%AD%B7%E7%90%86%E9%BB%83%E9%87%91%E6%9C%9F-030000799.html
https://www.bowtie.com.hk/blog/zh/%E8%89%B2%E6%96%91-%E5%8E%BB%E6%96%91%E7%99%82%E7%A8%8B/
https://www.discuss.com.hk/archiver/?tid-27890272.html


10202

In [ ]:
hk_key=[]

for keywords_S in hk_keywords:
  jaaKey = jaa.extract_tags(keywords_S, topK=15)
  for aly_keys in jaaKey:
    hk_key.append(aly_keys)

print(hk_key)

['打斑後護理', '斑後皮膚', '泛紅', '乾燥', '結焦', '問題', '加強', '激光', '效果', '改善', '打斑後護理', '斑後皮膚', '泛紅', '乾燥', '結焦', '問題', '加強', '激光', '效果', '改善', '問題', '出現', '療程', '皮膚', '色斑', '時刻', '發動', '膚質', '狀況', '若皮膚', '斑點', '膚色', '美麗', '面對', '醫美', '完斑', '一勞永逸', '預防', '防曬', '色斑', '色斑', '激光', '會誤', '無須', '作額', '膚護理', '防斑', '這種', '可謂', '大錯', '特錯', '其實', '適用', '現有', '生長', '皮膚', '出現', '反黑', '情況', '色斑', '色素', '激光', '面上', '一邊', '亦誤', '信網', '資訊', '以為', '打斑療', '程會', '而延', '反黑期', '會維持', '兩個', '時間', '機會', '因為', '膚色', '保養', '皮膚', '代謝', '使皮膚', '得宜', '早日', '加速', '大常見', '知清', '副作用', '激光', '泛紅', '高能量', '療程', '皮膚', '出現', '腫脹', '情況', '內回', '過份', '擔心', '皮秒', '激光', '星期', '不必', '正常', '乾燥', '療程後皮膚', '皮膚', '養份', '膚質', '變得', '繃緊', '出現', '脫皮', '做足', '保濕', '舒緩', '水份', '缺水', '增生', '黑色素', '水泡', '任其自然', '將肌底', '擊碎', '同時', '會令', '皮膚', '結痂', '繼而', '紅點', '現象', '脫落', '突起', '激光', '招護理', '貼士', '泛紅', '乾燥', '結焦', '助加', '強療程', '功效', '改善', '加強', '皮膚', '保濕', '皮膚', '肌膚', '敏感', '面膜', '天然', '首一', '洋甘菊', '療程後皮膚', '乾燥', '建議', '並選擇', '比較', '簡單', '保養品', '加強

In [ ]:
key_df = pd.DataFrame(hk_key, columns=['hk_key'])
hkKeyCount = key_df['hk_key'].value_counts()
hkKeyCount_df = pd.DataFrame(hkKeyCount.dropna().reset_index(name='hkKeyCount'))
hkValueSum = hkKeyCount_df['hkKeyCount'].sum()

hkValueSum
for i, p in enumerate(hkKeyCount_df['hkKeyCount']):
  hkKeyCount_df.loc[i, 'percent'] = (p/hkValueSum)*100
# hkKeyCount_df.index.values
display(hkKeyCount_df)

,index,hkKeyCount,percent
0,激光,1781,2.845457
1,脫毛,1156,1.846911
2,皮膚,790,1.262162
3,療程,648,1.035293
4,皮秒,528,0.843572
...,...,...,...
13519,傳染性,1,0.001598
13520,生疣,1,0.001598
13521,並盡,1,0.001598
13522,可將,1,0.001598


TW:

In [ ]:
hdr = {'User-Agent':'Mozilla/5.0', 'Accept': 'text/html, application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8'}
req = urllib.request.Request(url, headers=hdr)
html_page = urllib.request.urlopen(req, timeout=10)
html_soup = soup(html_page, "html.parser")
html_soup

tw_keywords = []

for i, url in enumerate(twlinks['link']):

  ###
  # if i < 50:
  print(url)
  hdr = {'User-Agent':'Mozilla/5.0', 'Accept': 'text/html, application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8'}
  try:
    req = urllib.request.Request(url, headers=hdr)
    html_page = urllib.request.urlopen(req, timeout=2)
    html_soup = soup(html_page, "html.parser")
    html_soup

    htmlTagToKeep = ['title', 'p', 'h1', 'h2', 'h3', 'h4']
    allTags = html_soup.findAll(htmlTagToKeep)

    for tag in allTags:
      tw_keywords.append(tag.text)
      # print(tag.text)
  except:
    print('Failed: '+url)
  ###

len(tw_keywords)

https://chantecaille.com.tw/blogs/le-magazine/leaser-treatment-skincare
https://review.com.tw/%E9%9B%B7%E5%B0%84%E8%A1%93%E5%BE%8C%E4%BF%9D%E9%A4%8A/
https://www.greenvines.com.tw/pages/blog-3-tips-for-laser-skin-care
https://www.realaesthetics.com.tw/Dr.Wang/knowledge/detail/15
https://shopee.tw/search?keyword=%E8%A1%93%E5%BE%8C%E4%BF%9D%E9%A4%8A%E5%93%81
https://www.beautywish.com.tw/BLOG/%E7%9A%AE%E7%A7%92%E8%A1%93%E5%BE%8C%E4%BF%9D%E9%A4%8A%E6%80%8E%E9%BA%BC%E5%81%9A%EF%BC%9F%E8%A1%93%E5%BE%8C%E4%BF%9D%E9%A4%8A%E5%93%81%E6%80%8E%E9%BA%BC%E6%8C%91%EF%BC%9F%E9%9B%B7%E5%B0%84%E5%89%8D%E5%85%88%E8%85%A6%E8%A3%9C%E4%BD%A0%E7%9A%84%E2%80%9D%E9%9B%B7%E5%B0%84%E6%99%BA%E5%95%86%E2%80%9D
Failed: https://www.beautywish.com.tw/BLOG/%E7%9A%AE%E7%A7%92%E8%A1%93%E5%BE%8C%E4%BF%9D%E9%A4%8A%E6%80%8E%E9%BA%BC%E5%81%9A%EF%BC%9F%E8%A1%93%E5%BE%8C%E4%BF%9D%E9%A4%8A%E5%93%81%E6%80%8E%E9%BA%BC%E6%8C%91%EF%BC%9F%E9%9B%B7%E5%B0%84%E5%89%8D%E5%85%88%E8%85%A6%E8%A3%9C%E4%BD%A0%E7%9A%84%E2%80%9D%E9%9B%B7%E5%

https://scitechvista.nat.gov.tw/Article/c000003/detail?ID=0e3f0bd9-f56c-4796-800f-d2b0f606beb1
https://beaute.usage.tw/spot.html
Failed: https://beaute.usage.tw/spot.html
https://www.remetw.com/after-laser-maintenance/
https://drugcoa.iwiki.tw/tag/%E9%9B%B7%E5%B0%84%E5%BE%8C%E5%A4%9A%E4%B9%85%E5%8F%AF%E4%BB%A5%E6%93%A6%E9%98%B2%E6%9B%AC/
https://health.ltn.com.tw/article/paper/442024
https://www.chimei.org.tw/main/cmh_department/59012/info/7550/A7550108.html
https://www.monafrema.com.tw/page/skincare009
Failed: https://www.monafrema.com.tw/page/skincare009
https://makeups.mediatagtw.com/article/%E9%9B%B7%E5%B0%84%E5%BE%8C%E5%A4%9A%E4%B9%85%E5%8F%AF%E4%BB%A5%E6%93%A6%E9%98%B2%E6%9B%AC
https://www.taic.mohw.gov.tw/public/ufile/f14a8a52e1b8e461d061d7dd7ca3ccf4.doc


https://www.ttv.com.tw/lohas/view/27837/554
https://www.auh.org.tw/NewsInfo/HealthEducationInfo?docid=869
https://zh-tw.facebook.com/chiupinchi/posts/%E9%9B%B7%E5%B0%84%E8%A1%93%E5%BE%8C%E7%9A%AE%E8%86%9A%E8%A6%81%E5%A6%82%E4%BD%95%E7%85%A7%E9%A1%A7%E4%BF%9D%E9%A4%8A%E9%95%B7%E6%98%A5%E6%9C%88%E5%88%8A%E8%A8%AA%E5%95%8F%E7%AF%80%E9%8C%84%E7%A8%BF%E5%8F%B0%E5%A4%A7%E9%86%AB%E9%99%A2%E9%9B%B2%E6%9E%97%E5%88%86%E9%99%A2%E7%9A%AE%E8%86%9A%E7%A7%91%E4%B8%BB%E4%BB%BB%E9%82%B1%E5%93%81%E9%BD%8A%E9%86%AB%E5%B8%AB%E8%A1%A8%E7%A4%BA%E7%9A%AE%E8%86%9A%E9%9B%B7%E5%B0%84%E8%A1%93%E5%BE%8C%E7%9A%84%E4%BF%9D%E9%A4%8A%E5%85%B6%E5%AF%A6%E4%B8%A6%E4%B8%8D%E5%A6%82%E6%83%B3%E5%83%8F%E4%B8%AD%E7%9A%84%E5%9B%B0%E9%9B%A3%E8%B7%9F%E4%B8%80%E8%88%AC%E5%9F%BA%E7%A4%8E%E4%BF%9D%E9%A4%8A%E5%9F%BA%E6%9C%AC%E4%B8%8A%E6%9C%89%E7%9B%B8%E4%BC%BC%E4%B9%8B%E8%99%95%E4%BD%86%E5%9C%A8%E5%AF%A6/643606832352882/
https://www.bigbeauty.com.tw/beautyinfo.php?nw=179
https://www.starclinic.com.tw/service_item.php?id=3
https://c

https://healthmedia.com.tw/main_detail.php?id=21991
https://article-consumer.fda.gov.tw/subject.aspx?subjectid=3&id=1404&parentid=72
https://m.pcone.com.tw/search?q=%E9%86%AB%E7%BE%8E%E8%A1%93%E5%BE%8C%E4%BF%9D%E9%A4%8A
https://www.paulaschoice.com.tw/learn.php?learn=2165
https://www.doredore.tw/BLOG/BlogDetail?cataId=1&blogId=362
https://feebee.com.tw/s/%E9%86%AB%E7%BE%8E%20%E8%A1%93%E5%BE%8C%E4%BF%9D%E9%A4%8A/
https://www.remetw.com/skincare-after-laser-treatment/
https://www.incres-lab.tw/post-treatment/
Failed: https://www.incres-lab.tw/post-treatment/
https://www.mbone.com.tw/news-view.php?ID=11
https://www.tw-aa.org/articledetail-1069.html
https://www.bishopclinic.com.tw/service_detail.asp?tid=28&cid=153&id=1419
https://p-paradise.com.tw/%E7%9A%AE%E7%A7%92%E9%9B%B7%E5%B0%84-%E9%99%A4%E6%96%91-%E9%99%A4%E5%88%BA%E9%9D%92/picosure-laser-safe-and-efficient
https://beautywithatwistblog.com/%E7%9A%AE%E7%A7%92%E5%8E%BB%E6%96%91/
https://www.iskin.com.tw/service/40/%E7%9A%AE%E7%A7%92%E9

6070

In [ ]:
tw_key=[]

for keywords_S in tw_keywords:
  jaaKey = jaa.extract_tags(keywords_S, topK=15)
  for aly_keys in jaaKey:
    tw_key.append(aly_keys)

print(tw_key)

['怎麼', '術後保養', '保養品', '瞭解', '大重點', '香緹卡', '雷射', '一次', '登入', '選單', '怎麼', '術後保養', '保養品', '瞭解', '大重點', '雷射', '一次', '雷射', '肌膚', '保養', '因為', '淨膚', '醫美療', '程是', '現在', '相當', '熱門', '有過', '經常', '狀況', '經驗', '先別', '術後保養', '雷射', '重要性', '肌膚', '雷射', '術後保養', '消費者', '特別', '乾癢', '紅腫', '會有', '細小傷', '結痂', '情況', '針對', '多數', '醫囑', '藥膏修', '術後保養', '怎麼', '大重點', '告訴', '保養', '保養品', '雷射', '編時', '不時', '耳聞', '錯誤', '觀念', '誤以', '完醫美', '肌膚', '趨近', '狀態', '而輕', '認為', '保養', '其實', '這是', '錯誤', '觀念', '護膚', '之間', '的關', '如並', '肩作戰', '戰友', '不論', '還是術', '小覷', '保養品', '術前', '保濕要', '產品', '使用', '噹噹', '術前', '周請', '盡量', '除毛', '角質', '狂曬', '太陽', '並加強', '保濕', '敷臉', '防曬產品', '盡力將', '保濕', '閱讀', '正確', '皮膚', '保養品', '推薦', '延伸', '方法', '術後保養', '仔細', '保養', '傷口', '而術後保養', '依據', '膚質', '與術後膚況', '選擇', '品外', '確且', '穩定術後敏弱', '肌膚', '這對', '將會', '我們', '將術後狀況', '人工', '臉部', '重則', '可見', '傷口', '此時', '照護', '不僅', '謹慎', '仔細', '醫生', '藥膏塗', '皮保護', '食鹽水', '進行', '傷口', '保養品', '色素', '使用', '避免', '保養', '美妝品', '影響', '產生', '反黑', '潛水', '酸類', '乾燥', '加強', '防曬', '保養品', '使用'

In [ ]:
key_df = pd.DataFrame(tw_key, columns=['tw_key'])
twKeyCount = key_df['tw_key'].value_counts()
twKeyCount_df = pd.DataFrame(twKeyCount.dropna().reset_index(name='twKeyCount'))
twValueSum = twKeyCount_df['twKeyCount'].sum()
for i, p in enumerate(twKeyCount_df['twKeyCount']):
  twKeyCount_df.loc[i, 'percent'] = (p/twValueSum)*100
display(twKeyCount_df)

,index,twKeyCount,percent
0,雷射,1210,2.924401
1,皮秒,495,1.196346
2,皮膚,416,1.005414
3,肌膚,405,0.978828
4,治療,348,0.841067
...,...,...,...
10952,NT1480,1,0.002417
10953,白煥膚,1,0.002417
10954,右邊,1,0.002417
10955,濕度,1,0.002417


Compare:

In [ ]:
domainNoLoc = masterNoDup[(masterNoDup['inTW']==False)&(masterNoDup['inHK']==False)].reset_index(drop=True)
hkKeySum = 0
twKeySum = 0

for i, url in enumerate(domainNoLoc['link']):

  ###
  # if i < 150:
  print(url)
  hdr = {'User-Agent':'Mozilla/5.0', 'Accept': 'text/html, application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8'}
  req = urllib.request.Request(url, headers=hdr)
  try:
    html_page = urllib.request.urlopen(req, timeout=2)
    html_soup = soup(html_page, "html.parser")

    for ii, k in enumerate(hkKeyCount_df['index']):
      cnt = html_soup.text.count(k)
      hkKeySum += hkKeyCount_df.loc[ii, 'percent']*cnt

    for ii, k in enumerate(twKeyCount_df['index']):
      cnt = html_soup.text.count(k)
      twKeySum += twKeyCount_df.loc[ii, 'percent']*cnt

    if (twKeySum<hkKeySum):
      domainNoLoc.loc[i, 'LocationByFilterText'] = 'hk'
      print('hk')
    elif(twKeySum>hkKeySum):
      domainNoLoc.loc[i, 'LocationByFilterText'] = 'tw'
      print('tw')
    else:
      domainNoLoc.loc[i, 'LocationByFilterText'] = 'unknown'
      print('unknown')
    hkKeySum = 0
    twKeySum = 0
  except:
    domainNoLoc.loc[i, 'LocationByFilterText'] = 'web died'
    print('This link cannot open')
  ###

https://perface.com/%E5%8E%BB%E6%96%91
hk
https://www.otandp.com/zh-hk/%E5%8D%9A%E5%AE%A2/%E9%86%AB%E5%AD%B8%E7%BE%8E%E5%AE%B9-%E6%BF%80%E5%85%89%E9%A1%9E%E5%9E%8B-%E5%8A%9F%E6%95%88
hk
https://www.union.org/new/tc_chi/health_info/health_articles_derm9.htm
hk
https://www.sorra.net/blog/2021/12/23/%E3%80%90%E5%81%9A%E5%AE%8C%E6%BF%80%E5%85%89%E5%BE%8C%E7%9A%AE%E8%86%9A%E5%8F%88%E7%B4%85%E5%8F%88%E4%B9%BE%E3%80%91%E4%B8%80%E6%96%87%E5%AD%B8%E6%9C%83%E6%BF%80%E5%85%89%E7%99%82%E7%A8%8B%E5%BE%8C/
tw
https://drhealthlab.com/%E8%B6%85%E2%BD%AA%E7%A7%92%E6%BF%80%E5%85%89%E8%AD%B7%E7%90%86-pico-treatments/
This link cannot open
https://www.mb-gghegf.com/chi/itemdetails.asp?id=FT017&rec=19&NewItem=&ItemClass1=%E7%BE%8E%E5%AE%B9%E8%AD%B7%E7%90%86&ItemClass2=%2D&ItemClass3=%2D&ItemClass4=%2D&viewresult=&viewresult2=&type=&searchword=&PriceCurrency=1
This link cannot open
https://www.sharebeautyclub.com/products/1064%E6%BF%80%E5%85%89%E5%8E%BB%E6%96%91%E8%AD%B7%E7%90%86
hk
https://www.modernbeauty

tw
https://www.momobeauty.com/tc/product?no=976
hk
http://asiapacific-beauty.com/Item/Show.asp?m=2&d=50
hk
https://kknews.cc/zh-hk/health/4nokmvg.html
hk
https://www.wowtcgdb.com/freckle_194220
hk
https://naturalgracebeauty.com/?page_id=1780
hk
https://www.plusmedispa.com/treatment/%E8%9C%82%E5%B7%A2%E7%9A%AE%E7%A7%92%E6%BF%80%E5%85%89-pico-laser-%E5%8E%BB%E6%96%91%E7%85%A5%E8%86%9A%E7%99%82%E7%A8%8B/
hk
https://amini.com.pl/%E7%9A%AE-%E7%A7%92-%E6%BF%80%E5%85%89-%E5%BE%8C-%E8%AD%B7%E7%90%86.html
tw
https://www.harpersbazaar.com/tw/beauty/skincare/g26003982/column-after-surgery-tips/
tw
https://pranazorg.nl/%E5%8E%BB-%E6%96%91-%E9%9B%B7-%E5%B0%84.html
This link cannot open
https://www.miyanlife.com/news/123004
hk
https://www.haodf.com/neirong/wenzhang/7944964976.html
hk
https://m.toutiao.com/i6694452689394205188/
tw
https://ppfocus.com/0/fa490a3f1.html
tw
https://new.qq.com/omn/20200912/20200912A0KNZG00.html
hk
https://zhinan.sogou.com/guide/d316513417522.htm?ch=zn.xqy.related.pc
hk
ht

In [ ]:
# domainNoLoc['LocationByFilterText']
print('HK:')
display(domainNoLoc.where(domainNoLoc['LocationByFilterText']=='hk')[['title', 'link', 'snippet']].dropna())
print('\n\n\nTW:')
display(domainNoLoc.where(domainNoLoc['LocationByFilterText']=='tw')[['title', 'link', 'snippet']].dropna())

HK:


,title,link,snippet
0,注意事項及去斑後護理方法！ - perFACE,https://perface.com/%E5%8E%BB%E6%96%91,於激光去斑療程後的恢復期間，治療部位有機會出現結焦現象，應特別注意，避免刺激皮膚，尤其洗臉或...
1,打laser懶人包：療程效用、風險及術後護理 - OT&P,https://www.otandp.com/zh-hk/%E5%8D%9A%E5%AE%A...,6. 激光治療後如何護理皮膚？ · 在治療後的首數天，皮膚可能較敏感，應避免任何傷害、摩擦和...
2,激光和彩光的風險及護理 - 仁安醫院,https://www.union.org/new/tc_chi/health_info/h...,激光治療後出現少許紅腫是正常現象，色斑位置可能會結痂，大約數天至一星期後痂口會完全脫去。由於...
6,1064激光去斑護理(網上付款享限定半價優惠),https://www.sharebeautyclub.com/products/1064%...,1064激光去斑護理有效鎖定並擊退色斑，刺激膠原蛋白增生，光澤膚質，改善皺紋，收細毛孔。10...
8,HairMax 激光生髮器頭帶41 + 頭皮護理洗髮水300ml ... - wishh!,https://wishh.com/index.php?route=product/prod...,HairMax 激光生髮器頭帶41 + 頭皮護理洗髮水300ml x2 + 頭皮護理護髮素2...
...,...,...,...
454,皮秒術後如何護理才能讓效果更持久？ - 壹讀,https://read01.com/zh-hk/7D3JkMP.html,做完蜂巢皮秒激光之後，治療部位都會有結痂，屬於正常現象（一般無需特殊處理），結痂後要等待自行...
455,Pico3second 皮秒激光去斑療程 - MoMo Beauty,https://www.momobeautyplus.com/tc/news_detail....,皮秒激光去斑同時將震波傳至肌膚深層，啟動修復機制，促進膠原蛋白嘅更新與 ... 蜂巢皮秒激光...
457,做完皮秒后，请做好这些护理工作！ - 新浪,https://k.sina.cn/article_6358793656_17b037db8...,个别皮肤敏感的人在皮秒激光治疗后可能会出现一些程度的红肿现象，这是皮肤受激光后的热效应导致的...
458,皮秒祛斑後結痂如何護理皮秒激光祛斑的原理- 整形- 靚麗少女網,https://llsnw.com/meir/zhengxing/rx1dj.html,皮秒祛斑後結痂如何護理皮秒激光祛斑的原理説明：在我們平時生活中有很多人都去做了皮秒，皮秒是一...





TW:


,title,link,snippet
3,【做完激光後皮膚又紅又乾?】一文學會激光療程後的護理方法！,https://www.sorra.net/blog/2021/12/23/%E3%80%9...,因皮秒激光(Pico Laser)比傳統激光的速度更快，有效把皮膚內的黑色素快速震碎，並隨著...
7,二氧化碳激光嫩膚護理 - 現代美容中心,https://www.modernbeautysalon.com/product/%E4%...,二氧化碳激光嫩膚護理 ... 現有顧客如在六個月內完成使用所有購買的療程，同時沒有購買新療程...
10,Collagen+ Medical Laser Skin Care Centre 醫學激光皮...,https://www.instagram.com/explore/locations/93...,26 Posts - See Instagram photos and videos tak...
11,皮秒激光後護理,http://cbmv.cl/%E7%9A%AE-%E7%A7%92-%E6%BF%80%E...,皮秒激光後護理. 進行激光去斑療程後的護膚程序為何? 若醫生有提供藥膏，就要按時於傷口塗上。...
13,激光護理 - BIO Beauty Spa,https://www.biobeautyspa.com/laser-treatments-chi,"激光嫩膚美白去辦面部護理, 激光去紋收緊頸部護理. Fotona QX Max是市面上最先進..."
...,...,...,...
447,激光、雷射、皮秒等医美手术后，如何进行肌肤护理 - 今日头条,https://m.toutiao.com/i6694452689394205188/,激光、雷射、皮秒等医美手术后，如何进行肌肤护理 · Q1: 是否要购入医院推荐的专门的医美术...
448,皮秒雷射術後不想毀容?你一定不能這樣做! - 人人焦點,https://ppfocus.com/0/fa490a3f1.html,這是皮秒雷射後護理中的重中之重啊，姐妹們，一定一定一定要防曬！如果長時間外出，我們建議打傘戴...
451,醫美【蜂巢皮秒雷射】Picosecond 紀錄感想(上) 多久打一次 ...,https://ianchiu1107.pixnet.net/blog/post/40398...,神奇的蜂巢皮秒雷射幾天前陪著好友去南京西路的一間醫美診所咨詢關於肉毒桿菌的相關療程時，剛好診...
456,激光、雷射、皮秒等醫美手術後，如何進行肌膚護理| 陸劇吧,https://lujuba.cc/zh-hant/308431.html,敏感肌、乾燥肌和油痘肌，術後的護理是完全不一樣的。 激光、雷射、皮秒等醫美手術後，如何進行肌...


In [ ]:
pd.DataFrame(domainNoLoc.where(domainNoLoc['LocationByFilterText']=='hk')[['title', 'link', 'snippet']].dropna()).to_csv('allHK.csv')
pd.DataFrame(domainNoLoc.where(domainNoLoc['LocationByFilterText']=='tw')[['title', 'link', 'snippet']].dropna()).to_csv('allTW.csv')

hkKeyCount_df.to_csv('hkKey.csv')
twKeyCount_df.to_csv('twKey.csv')